In [1]:
import os
from os.path import join
import pandas as pd
import datetime
import matplotlib.pyplot as plt 
import seaborn as sns

In [2]:
train=join('dataset','train.csv')
test=join('dataset','test.csv')

train=pd.read_csv(train) #data[0:1205]
test=pd.read_csv(test)  #data[1205:]

In [3]:
train.tail()

,일자,요일,본사정원수,본사휴가자수,본사출장자수,본사시간외근무명령서승인건수,현본사소속재택근무자수,조식메뉴,중식메뉴,석식메뉴,중식계,석식계
1200,2021-01-20,수,2983,75,198,4,391.0,모닝롤/페퍼로니피자 우유/주스 계란후라이/찐계란 크루통크림스프/흑미밥 아귀지리 마늘...,쌀밥/흑미밥/찰현미밥 아욱국 수제함박스테이크 견과류마카로니범벅 생깻잎지 단호박물김치...,김치볶음밥 미니쫄우동*맛살튀김 브로콜리깨소스무침 계란후라이 고들빼기무침 겉절이김치,1093.0,421.0
1201,2021-01-21,목,2983,92,231,462,351.0,모닝롤/생크림단팥빵 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 떡국 해물땡굴소스볶...,쌀밥/수수밥/찰현미밥 냉이된장국 동파육 봄동전 청경채/버섯숙회*초장 무생채 양상추샐...,흑미밥 쇠고기무국 삼치양념구이 비엔나채소볶음 숙주나물당근무침 포기김치,832.0,353.0
1202,2021-01-22,금,2983,255,248,1,303.0,모닝롤/BLT샌드위치 우유/주스 계란후라이/찐계란 흑임자죽/흑미밥 바지락살국 두부조...,전주비빔밥*약고추장 계란파국 요거닭 올방개묵무침 파프리카해초무침 포기김치 양상추샐러...,흑미밥 수제비국 수제맛쵸킹탕수육 유부채소겨자냉채 참나물무침 갓김치/겉절이김치,579.0,217.0
1203,2021-01-25,월,2983,107,153,616,327.0,모닝롤/호박고구마오븐구이 우유/주스 계란후라이/찐계란 누룽지탕/흑미밥 감자양파국 분...,쌀밥/흑미밥/찰현미밥 전주식콩나물해장국 돈육간장불고기 깐풍연근 연두부*달래양념장 봄...,흑미밥 열무된장국 장어강정*데리야끼소스 깻잎쌈*생강채 오이선 포기김치,1145.0,502.0
1204,2021-01-26,화,2983,69,183,551,362.0,모닝롤/야채샌드 우유/주스 계란후라이/찐계란 참치죽/흑미밥 홍합탕 애호박새우젓볶음 ...,쌀밥/귀리밥/찰현미밥 들깨미역국 교촌간장치킨 옥수수콘치즈구이 가지고추장무침 포기김치...,(New)할라피뇨멸치주먹밥 잔치국수 수제고기육전 쑥갓나물 양파초절임 깍두기,1015.0,480.0


# train data 와 test data 합치기
- 라벨인코딩에서 빠진 없이 인코딩 해주기 위해서 data를 합침

In [3]:
#encoding을 위해서 test train 합치기
data = pd.concat([train,test],axis=0)
data=data.reset_index()

In [4]:
data=data.drop(['index'],axis=1)

In [5]:
#column명 바꾸기
data.columns = ['date', 'dow', 'total', 'day-off', 'bustrip', 'overtime', 'remote', 
                      'breakfast', 'lunch', 'dinner', 'target_lunch', 'target_dinner']

In [6]:
#날짜 데이터
data['date']=pd.to_datetime(data['date'])
data['dow']=pd.to_datetime(data["date"]).dt.weekday
data["year"]=pd.to_datetime(data["date"]).dt.year
data["month"]=pd.to_datetime(data["date"]).dt.month
data["day"]=pd.to_datetime(data["date"]).dt.day

#전체 근무자 계산
data['office'] = data['total'] - data['day-off'] - data['bustrip'] - data['remote'] + data['overtime']

In [7]:
#메뉴 인코딩
lunch = []
for day in range(len(data)):
    tmp = data.iloc[day, 8].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    lunch.append(tmp)

    
dinner = []
for day in range(len(data)):
    tmp = data.iloc[day, 9].split(' ') # 공백으로 문자열 구분 
    tmp = ' '.join(tmp).split()    # 빈 원소 삭제

    search = '('   # 원산지 정보는 삭제
    for menu in tmp:
        if search in menu:
            tmp.remove(menu)
    
    dinner.append(tmp)

In [8]:
lunch = pd.DataFrame(lunch)
dinner = pd.DataFrame(dinner)

lunch=lunch.iloc[:,0:5]
lunch.columns = ['lunch_bob','lunch_gok','lunch_side1','lunch_side2','lunch_side3']

dinner=dinner.iloc[:,0:5]
dinner.columns = ['dinner_bob','dinner_gok','dinner_side1','dinner_side2','dinner_side3']

In [9]:
data=pd.concat([data,lunch,dinner],axis=1)
data=data.drop(['date','breakfast','lunch','dinner'],axis=1)

# 휴일 column 생성

In [10]:
data['special_day'] = 0

In [11]:
self=(data.dinner_bob=='*자기계발의날*') | (data.dinner_bob =='자기계발의날') |(data.dinner_bob =='자기개발의날') |(data.dinner_gok =='자기개발의날') | (data.dinner_gok == '자기계발의날') |(data.dinner_bob=='＜자기')
holi=(data.dinner_bob=='*') & (data.dinner_gok!='자기개발의날') &(data.dinner_gok!='자기계발의날')
family= (data.dinner_bob =='가정의날')

In [12]:
data.loc[self,'special_day']=1
data.loc[holi,'special_day']=2
data.loc[family,'special_day']=3
## 모두 none 인 데아터는 인덱싱 하지 못함

In [13]:
data[data['target_dinner'] == 0]

,dow,total,day-off,bustrip,overtime,remote,target_lunch,target_dinner,year,month,...,lunch_gok,lunch_side1,lunch_side2,lunch_side3,dinner_bob,dinner_gok,dinner_side1,dinner_side2,dinner_side3,special_day
204,2,2689,68,207,0,0.0,1109.0,0.0,2016,11,...,가쯔오장국,치킨핑거*요거트D,감자샐러드,오복지무침,*,None,None,None,None,2
224,2,2705,166,225,0,0.0,767.0,0.0,2016,12,...,가쯔오장국,미트볼케찹조림,꽃맛살샐러드,군고구마,*,None,None,None,None,2
244,2,2697,79,203,0,0.0,720.0,0.0,2017,1,...,맑은국,유린기,개성감자만두,오이사과무침,*,None,None,None,None,2
262,2,2632,75,252,0,0.0,1065.0,0.0,2017,2,...,유부장국,생선까스*탈탈소스,파스타샐러드,마늘쫑볶음,*,None,None,None,None,2
281,2,2627,53,235,0,0.0,953.0,0.0,2017,3,...,돈육김치찌개,유린기,비엔나볶음,세발나물,*,None,None,None,None,2
306,2,2626,45,304,0,0.0,835.0,0.0,2017,4,...,맑은국,오징어튀김,견과류조림,하와이안샐러드,*,None,None,None,None,2
327,2,2637,43,265,0,0.0,910.0,0.0,2017,5,...,가쯔오장국,탕수만두,콥샐러드,오이지무침,자기계발의날,None,None,None,None,1
346,2,2648,58,259,0,0.0,745.0,0.0,2017,6,...,얼갈이된장국,삼치구이,잡채,아삭고추무침,*자기계발의날*,None,None,None,None,1
366,2,2839,254,246,0,0.0,797.0,0.0,2017,7,...,미소장국,파스타샐러드,소세지오븐구이,오렌지,가정의날,None,None,None,None,3
392,4,2642,177,303,45,0.0,663.0,0.0,2017,9,...,시래기국,훈제오리구이,두부구이*양념장,쌈무/양파절임,*,None,None,None,None,2


In [14]:
data['dinner_gok']=data['dinner_gok'].fillna('missing')
data['dinner_bob']=data['dinner_bob'].fillna('missing')
data['dinner_side1']=data['dinner_side1'].fillna('missing')
data['dinner_side2']=data['dinner_side2'].fillna('missing')
data['dinner_side3']=data['dinner_side3'].fillna('missing')

# 비율 column 생성

In [15]:
data['dinner_rate'] = data['target_dinner']/data['office']
data['lunch_rate'] = data['target_lunch']/data['office']

# 메뉴명 라벨인코딩

In [16]:
##라벨인코딩
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le

LabelEncoder()

In [17]:
cols=['lunch_bob','lunch_gok','lunch_side1','lunch_side2','lunch_side3','dinner_bob','dinner_gok','dinner_side1','dinner_side2','dinner_side3','lunch_bob']

In [18]:
for col in cols:
    label = data[col]
    le.fit(label)
    label_encoded = le.transform(label)
    data=data.drop([col],axis=1)
    l_e_df = pd.DataFrame(label_encoded,columns = [col])
    data = pd.concat([data,l_e_df],axis=1)

In [19]:
data

,dow,total,day-off,bustrip,overtime,remote,target_lunch,target_dinner,year,month,...,lunch_gok,lunch_side1,lunch_side2,lunch_side3,dinner_bob,dinner_gok,dinner_side1,dinner_side2,dinner_side3,lunch_bob
0,0,2601,50,150,238,0.0,1039.0,331.0,2016,2,...,199,246,39,399,74,238,366,171,19,52
1,1,2601,50,173,319,0.0,867.0,560.0,2016,2,...,29,9,203,109,122,195,360,358,180,52
2,2,2601,56,180,111,0.0,1017.0,573.0,2016,2,...,261,355,411,21,74,262,464,54,228,71
3,3,2601,104,220,355,0.0,978.0,525.0,2016,2,...,157,327,253,248,41,227,184,68,161,52
4,4,2601,278,181,34,0.0,925.0,330.0,2016,2,...,78,111,395,393,74,258,96,18,446,52
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1250,0,2973,125,174,704,331.0,NaN,NaN,2021,4,...,159,343,379,4,137,66,17,87,427,57
1251,1,2973,76,170,636,364.0,NaN,NaN,2021,4,...,166,155,82,99,115,251,301,215,86,40
1252,2,2973,96,214,1,334.0,NaN,NaN,2021,4,...,42,99,262,326,32,183,277,503,11,57
1253,3,2973,105,238,509,324.0,NaN,NaN,2021,4,...,82,4,398,445,137,196,281,460,277,50


# test train data 나누기

In [20]:
train=data[0:1205]
test=data[1205:]
test=test.reset_index()
test=test.drop(['index'],axis=1)

# modeling

In [21]:
from sklearn.metrics import mean_absolute_error

#MAE 평가 함수
def evaluate_regr(y,pred):
    mae_val = mean_absolute_error(y,pred)
    print('MAE:{0:.3F}'.format(mae_val))

In [22]:
#corss validation예측 함수
def get_model_predict(model,X_train,X_test,y_train,y_test,is_expm1=False):
    model.fit(X_train,y_train)
    pred=model.predict(X_test)
    if is_expm1:
        y_test = np.expm1(y_test)
        pred = np.expm1(pred)
    print('###',model.__class__.__name__,'###')
    evaluate_regr(y_test,pred)

In [23]:
#ligthgbm model
from lightgbm import LGBMRegressor

lgbm_reg = LGBMRegressor(n_estimators=500)
lgbm_params = {'n_estimators':[1000]}
lgbm_reg =LGBMRegressor(n_estimators=1000, learning_rate = 0.05 ,num_leaves=4
                       ,subsample=0.6,colsample_bytree=0.4,reg_lambda=10,n_jobs=-1)

## 단순 인원수 예측

In [24]:
from sklearn.model_selection import train_test_split ,GridSearchCV
#corss validation
y1_target = train['target_dinner']
y2_target = train['target_lunch']

X_features = train.drop(['target_dinner','target_lunch'],axis=1,inplace=False)

X1_train,X1_test,y1_train,y1_test = train_test_split(X_features,y1_target, test_size=0.3,random_state=0)
X2_train,X2_test,y2_train,y2_test = train_test_split(X_features,y2_target, test_size=0.3,random_state=0)

In [25]:
get_model_predict(lgbm_reg,X1_train.values,X1_test.values,y1_train.values,y1_test.values,is_expm1=False)
get_model_predict(lgbm_reg,X2_train.values,X2_test.values,y2_train.values,y2_test.values,is_expm1=False)

### LGBMRegressor ###
MAE:8.919
### LGBMRegressor ###
MAE:15.969


## 비율 예측

In [26]:
from sklearn.model_selection import train_test_split ,GridSearchCV
#corss validation
y1_target = train['dinner_rate']*train['office']
y2_target = train['lunch_rate']*train['office']

X_features = train.drop(['dinner_rate','lunch_rate'],axis=1,inplace=False)

X1_train,X1_test,y1_train,y1_test = train_test_split(X_features,y1_target, test_size=0.3,random_state=0)
X2_train,X2_test,y2_train,y2_test = train_test_split(X_features,y2_target, test_size=0.3,random_state=0)

In [27]:
get_model_predict(lgbm_reg,X1_train.values,X1_test.values,y1_train.values,y1_test.values,is_expm1=False)
get_model_predict(lgbm_reg,X2_train.values,X2_test.values,y2_train.values,y2_test.values,is_expm1=False)

### LGBMRegressor ###
MAE:6.077
### LGBMRegressor ###
MAE:12.899


## 공휴일 영향 X (단순 count)

In [28]:
from sklearn.model_selection import train_test_split ,GridSearchCV
#corss validation
y1_target = train['target_dinner']
y2_target = train['target_lunch']

X_features = train.drop(['target_dinner','target_lunch','special_day'],axis=1,inplace=False)

X1_train,X1_test,y1_train,y1_test = train_test_split(X_features,y1_target, test_size=0.3,random_state=0)
X2_train,X2_test,y2_train,y2_test = train_test_split(X_features,y2_target, test_size=0.3,random_state=0)

In [29]:
get_model_predict(lgbm_reg,X1_train.values,X1_test.values,y1_train.values,y1_test.values,is_expm1=False)
get_model_predict(lgbm_reg,X2_train.values,X2_test.values,y2_train.values,y2_test.values,is_expm1=False)

### LGBMRegressor ###
MAE:9.516
### LGBMRegressor ###
MAE:15.934


## 공휴일 영향 X (비율)

In [30]:
from sklearn.model_selection import train_test_split ,GridSearchCV
#corss validation
y1_target = train['dinner_rate']*train['office']
y2_target = train['lunch_rate']*train['office']

X_features = train.drop(['dinner_rate','lunch_rate','special_day'],axis=1,inplace=False)

X1_train,X1_test,y1_train,y1_test = train_test_split(X_features,y1_target, test_size=0.3,random_state=0)
X2_train,X2_test,y2_train,y2_test = train_test_split(X_features,y2_target, test_size=0.3,random_state=0)

In [31]:
get_model_predict(lgbm_reg,X1_train.values,X1_test.values,y1_train.values,y1_test.values,is_expm1=False)
get_model_predict(lgbm_reg,X2_train.values,X2_test.values,y2_train.values,y2_test.values,is_expm1=False)

### LGBMRegressor ###
MAE:6.944
### LGBMRegressor ###
MAE:13.982
